In [ ]:
import pandas as pd
# fb = pd.read_csv('fb.csv')
# twitter = pd.read_csv('twitter.csv')
# qualtrics = pd.read_csv('qualtrics.csv')

qualtrics

In [60]:
round(twitter['politics']).unique()

array([4., 5., 1., 7., 6., 2., 3.])

In [4]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import abc
import re
import itertools
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import csv

def proportion_unique_words(topics, topk=10):
    if topk > len(topics[0]):
        raise Exception('Words in topics are less than '+str(topk))
    else:
        unique_words = set()
        for topic in topics:
            unique_words = unique_words.union(set(topic[:topk]))
        puw = len(unique_words) / (topk * len(topics))
        return puw

class Coherence(abc.ABC):
    def __init__(self, topics, texts):
        self.topics = topics
        self.texts = texts
        self.dictionary = Dictionary(self.texts)

    @abc.abstractmethod
    def score(self):
        pass

class CoherenceNPMI(Coherence):
    def __init__(self, topics, texts):
        super().__init__(topics, texts)

    def score(self, topk=10, per_topic=False):
        if topk > len(self.topics[0]):
            raise Exception('Words in topics are less than topk')
        else:
            npmi = CoherenceModel(
                topics=self.topics, texts=self.texts,
                dictionary=self.dictionary,
                coherence='c_npmi', topn=topk)
            if per_topic:
                return npmi.get_coherence_per_topic()
            else:
                return npmi.get_coherence()



dataset_model_scores = {}

# for dataset in ['yelp_reviews', 'fb', 'twitter', 'qualtrics', '20_newsgroup']:
for dataset in ['fb']:
    print(dataset)

    df = pd.read_csv(dataset + ".csv")
    if dataset == 'amazon':
        documents = [line.strip() for line in (df['text']) if not isinstance(line, float)]
    else:
        documents = [line.strip() for line in (df['message']) if not isinstance(line, float)]

    model_scores = {} # tuple of model : (avg diversity, avg coherence)

    for model in ['GensimLDA', 'Mallet_LDA', 'CTM', 'BERTopic', 'NMF']:
        print(model)

        diversities = []
        coherences = []

        for run in range(1, 6):
            print(run)

            topics = []
            with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
                reader = csv.reader(f)
                for row in reader:
                    topic_list = [item.strip() for item in row if item.strip()]
                    topics.append(topic_list)

            warnings.filterwarnings('ignore')
            topic_diversity = proportion_unique_words(topics)
            tokenizer = lambda s: re.findall( '\w+', s.lower() )
            texts = [tokenizer(t) for t in  documents]
            coherence = CoherenceNPMI(texts=texts, topics=topics).score()

            diversities.append(topic_diversity)
            coherences.append(coherence)
        
        model_scores[model] = (np.mean(diversities), np.mean(coherences))
        print(np.mean(diversities))
        print(np.mean(coherences))
    
    dataset_model_scores[dataset] = model_scores

fb
GensimLDA
1


Python(21290) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(21292) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(21294) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(21296) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(21298) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(21300) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(21303) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


2


In [3]:
dataset = 'bbc_news'
model = 'GensimLDA'

df = pd.read_csv(dataset + ".csv")
documents = [line.strip() for line in (df['message']) if not isinstance(line, float)]

print(model)

diversities = []
coherences = []

for run in range(1, 6):
    print(run)

    topics = []
    with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            topic_list = [item.strip() for item in row if item.strip()]
            topics.append(topic_list)

    warnings.filterwarnings('ignore')
    topic_diversity = proportion_unique_words(topics)
    tokenizer = lambda s: re.findall( '\w+', s.lower() )
    texts = [tokenizer(t) for t in  documents]
    coherence = CoherenceNPMI(texts=texts, topics=topics).score()

    diversities.append(topic_diversity)
    coherences.append(coherence)

print(np.mean(diversities))
print(np.mean(coherences))

GensimLDA
1
2
3
4
5
0.489
-0.048823966202612354
